<!-- TABS -->
# Build multimodal embedding models

In [9]:
# <tab: Text>
from superduperdb.ext.sentence_transformers import SentenceTransformer

# Load the pre-trained sentence transformer model
superdupermodel = SentenceTransformer(identifier='all-MiniLM-L6-v2')

In [10]:
# <testing: >
import numpy as np
from PIL import Image

embeddings = superdupermodel.predict_one('some text')

In [11]:
# <tab: Image>
import torch
import clip
from torchvision import transforms
from superduperdb.ext.torch import TorchModel

class CLIPVisionEmbedding:
    def __init__(self):
        # Load the CLIP model
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.preprocess = clip.load("RN50", device=self.device)
        
    def preprocess(self, image):
        # Load and preprocess the image
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        return image
        
model = CLIPVisionEmbedding()
superdupermodel = TorchModel(identifier='clip-vision', object=model.model, preprocess=model.preprocess, forward_method='encode_image')

In [12]:
# <tab: Text-2-Image>

import torch
import clip
from torchvision import transforms
from superduperdb import Model
from superduperdb.ext.torch import TorchModel

class CLIPTextEmbedding:
    def __init__(self):
        # Load the CLIP model
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, _ = clip.load("RN50", device=self.device)
        
    def __call__(self, text):
        features = clip.tokenize([text])
        return self.model.encode_text(features)
        
model = CLIPTextEmbedding()
superdupermodel_text = Model(identifier='clip-text', object=model)

class CLIPVisionEmbedding:
    def __init__(self):
        # Load the CLIP model
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.preprocess = clip.load("RN50", device=self.device)
        
    def preprocess(self, image):
        # Load and preprocess the image
        image = self.preprocess(image).unsqueeze(0).to(self.device)
        return image
        
model = CLIPVisionEmbedding()
superdupermodel_image = TorchModel(identifier='clip-vision', object=model.model, preprocess=model.preprocess, forward_method='encode_image')

In [13]:
# <testing: >
import numpy as np
from PIL import Image

embeddings = superdupermodel_image.predict_one(Image.fromarray(np.ones((256,256,3)).astype(np.uint8)))

 2024-Mar-27 14:47:07.83| INFO     | kartiks-MacBook-Air.local| superduperdb.components.component:344  | Initializing TorchModel : clip-vision
 2024-Mar-27 14:47:07.83| INFO     | kartiks-MacBook-Air.local| superduperdb.components.component:347  | Initialized  TorchModel : clip-vision successfully


In [14]:
# <testing: >
import numpy as np
from PIL import Image

embeddings = superdupermodel_text.predict_one('some text')

 2024-Mar-27 14:47:07.92| INFO     | kartiks-MacBook-Air.local| superduperdb.components.component:344  | Initializing ObjectModel : clip-text
 2024-Mar-27 14:47:07.92| INFO     | kartiks-MacBook-Air.local| superduperdb.components.component:347  | Initialized  ObjectModel : clip-text successfully


In [2]:
# <tab: Audio>
!pip install librosa
import librosa
import numpy as np
from superduperdb import Model

def audio_embedding(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    return mfccs
superdupermodel = Model(identifier='my-model-audio', object=audio_embedding)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 1.6 MB/s eta 0:00:001.6 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.5/390.5 kB 9.3 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
# <testing:>
import wave
import struct

sample_rate = 44100 
duration = 1 
frequency = 440
amplitude = 0.5

# Generate the sine wave
num_samples = int(sample_rate * duration)
t = np.linspace(0, duration, num_samples, False)
signal = amplitude * np.sin(2 * np.pi * frequency * t)

# Open a new WAV file
output_file = 'dummy_audio.wav'
wav_file = wave.open(output_file, 'w')

# Set the parameters for the WAV file
nchannels = 1  # Mono audio
sampwidth = 2  # Sample width in bytes (2 for 16-bit audio)
framerate = sample_rate
nframes = num_samples

# Set the parameters for the WAV file
wav_file.setparams((nchannels, sampwidth, framerate, nframes, 'NONE', 'not compressed'))

# Write the audio data to the WAV file
for sample in signal:
    wav_file.writeframes(struct.pack('h', int(sample * (2 ** 15 - 1))))

# Close the WAV file
wav_file.close()

# Test
superdupermodel.predict_one(output_file)